<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
from pathlib import Path 
current_path = Path().resolve()
abs_path = str(current_path.parent)
sys.path.append(abs_path)

import basedosdados as bd
import pymove as pm
import gtfs_kit as gk
import pandas as pd
from shapely.geometry import Point
from functools import partial
from shapely.ops import transform
import geopandas as gpd
import pyproj

from pymove.utils import log

In [2]:
log.set_verbosity('ERROR')

In [3]:
from rGTFS import simple, io, utils
from rGTFS.simple import *

In [4]:
gps_path = '../data/raw/gps_brt-2021-04-07.csv'
gtfs_path = '../data/output/gtfs_brt.zip'
rgtfs_path = '../data/output/rgtfs_brt.zip'

In [2]:
# Load GPS

In [351]:
gps = bd.read_sql(
    """
    SELECT latitude, longitude, timestamp_captura as datetime, linha as route_id, codigo as vehicle_id
    FROM `rj-smtr.dashboard_monitoramento_brt.registros_filtrada` 
    WHERE data = DATE '2021-04-07'
    AND placa is not null
    """,
    billing_project_id='rj-smtr'
)

Downloading: 100%|██████████| 117729/117729 [00:05<00:00, 22551.03rows/s]


In [359]:
gps.to_csv(gps_path, index=False)

In [616]:
stop_buffer_radius = 100
ignore_vehicles_with_wrong_route_id = True

gps = io.read_gps(gps_path)
gps = clean_gps(gps)

gtfs = io.read_gtfs(gtfs_path, "km")
first_last_stops = get_first_last_stops(gtfs)
first_last_stops = add_buffer_around_stops(
    first_last_stops, gtfs, stop_buffer_radius
)
first_last_stops = gpd.GeoDataFrame(first_last_stops, geometry="stop_buffer")

trips = get_trips(first_last_stops, gps, gtfs, ignore_vehicles_with_wrong_route_id)

In [16]:
trips, rgtfs = simple.main(gtfs_path, gps_path, rgtfs_path)

In [635]:
import datetime

In [655]:
rgtfs = simple.trips_to_gtfs(trips, gtfs)

In [656]:
rgtfs.frequencies

,trip_id,start_time,end_time,headway_secs,exact_times
0,10_0_U,01:00:00,02:00:00,3600.0,NaN
1,10_0_U,03:00:00,04:00:00,1800.0,NaN
2,10_0_U,04:00:00,05:00:00,3600.0,NaN
3,10_0_U,05:00:00,06:00:00,3600.0,NaN
4,10_0_U,07:00:00,08:00:00,720.0,NaN
...,...,...,...,...,...
413,LECD33_1_U,15:00:00,16:00:00,3600.0,NaN
414,LECD33_1_U,16:00:00,17:00:00,3600.0,NaN
415,LECD33_1_U,17:00:00,18:00:00,3600.0,NaN
416,LECD33_1_U,20:00:00,21:00:00,3600.0,NaN


In [11]:
import plotly.express as px

In [24]:
gtfs = io.read_gtfs(gtfs_path, "km")

In [25]:
rgtfs = io.read_gtfs(rgtfs_path, "km")

In [33]:
def hourly_trips(feed, label):
    f = deepcopy(feed.frequencies)
    f['n_trips'] = 3600 / f['headway_secs'] 
    f =f.groupby(['start_time'])['n_trips'].sum().reset_index()
    f['label'] = label
    return f

In [41]:
a = pd.concat([r,g]).pivot_table(index='start_time', columns='label', values='n_trips')
int(abs(a['Executado'] - a['Planejado']).sum())

2059

In [35]:
r = hourly_trips(rgtfs, 'Executado')
g = hourly_trips(gtfs, 'Planejado')
fig = px.line(pd.concat([r,g]), x='start_time', y='n_trips', color='label')

fig.update_layout(
    template='simple_white',
    yaxis_title='Número de Viagens',
    title='Número de Viagens BRT em 2021-04-07 (Planejado vs. Executado)',
    hovermode="x"
)

fig.show()



In [666]:
week = rgtfs.get_first_week()
week

['20210104',
 '20210105',
 '20210106',
 '20210107',
 '20210108',
 '20210109',
 '20210110']

In [674]:
trip_stats = gtfs.compute_trip_stats()

In [675]:
gtfs.compute_feed_time_series(trip_stats, ['20210407'], freq='6H')

indicator,num_trip_ends,num_trip_starts,num_trips,service_distance,service_duration,service_speed
datetime,,,,,,
2021-04-07 00:00:00,10,31,31,605.349089,43.916667,13.784040
2021-04-07 06:00:00,29,8,29,318.219731,22.000000,14.464533
2021-04-07 12:00:00,0,0,0,0.000000,0.000000,0.000000
2021-04-07 18:00:00,0,0,0,0.000000,0.000000,0.000000
